In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# first neural network with keras 
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense,Activation
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
np.random.seed(0)

Using TensorFlow backend.
C:\Users\kahme\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\kahme\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\kahme\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\kahme\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [2]:
ML = pd.read_csv('2017 Infant Linked Data_ML.csv')
ML.head()

,Mothers Single Race Code,Average Age of Mother Code,Mothers Education Code,Delivery Method Code,Deaths,Births,DeathRate
0,5,17.0,2,1,12,846,14.18
1,5,17.0,2,1,15,1050,14.29
2,5,17.0,2,1,27,2091,12.91
3,5,22.0,2,1,13,916,14.19
4,5,22.0,2,1,17,1238,13.73


In [22]:
ML.dtypes

Mothers Single Race Code        int64
Average Age of Mother Code    float64
Mothers Education Code          int64
Delivery Method Code            int64
Deaths                          int64
Births                          int64
DeathRate                     float64
dtype: object

In [46]:
X = ML[["Average Age of Mother Code", "Deaths","Births","DeathRate"]]
y = ML[[ "Mothers Single Race Code","Mothers Education Code", "Delivery Method Code"]]
y

,Mothers Single Race Code,Mothers Education Code,Delivery Method Code
0,5,2,1
1,5,2,1
2,5,2,1
3,5,2,1
4,5,2,1
...,...,...,...
69,3,2,2
70,3,2,2
71,3,2,2
72,2,2,2


In [74]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
y = ohe.fit_transform(y).toarray()


C:\Users\kahme\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [53]:
#Normalizing the data
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)

In [54]:
# We create the preprocessing pipelines for both numeric and categorical data.
numeric_features = ['Average Age of Mother Code', 'Deaths','Births','DeathRate']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_features = ['Mothers Single Race Code','Mothers Education Code', 'Delivery Method Code']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

In [55]:
print("Shape: ", X.shape, y.shape)

Shape:  (74, 4) (74, 10)


In [75]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [57]:
# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', LogisticRegression())])

#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


clf.fit(X_train, y_train)
#print("model score: %.3f" % clf.score(X_test, y_test))

ValueError: Specifying the columns using strings is only supported for pandas DataFrames

In [41]:
param_grid = {
    'preprocessor__num__imputer__strategy': ['mean', 'median'],
    'classifier__C': [0.1, 1.0, 10, 100],
}

grid_search = GridSearchCV(clf, param_grid, cv=10)
grid_search.fit(X_train, y_train)

print(("best logistic regression from grid search: %.3f"
       % grid_search.score(X_test, y_test)))

C:\Users\kahme\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:528: FutureWarning: From version 0.22, errors during fit will result in a cross validation score of NaN by default. Use error_score='raise' if you want an exception raised or error_score=np.nan to adopt the behavior from version 0.22.
  FutureWarning)


ValueError: 'Mothers Single Race Code' is not in list

In [24]:
ML.values

array([[5.00000e+00, 1.70000e+01, 2.00000e+00, 1.00000e+00, 1.20000e+01,
        8.46000e+02, 1.41800e+01],
       [5.00000e+00, 1.70000e+01, 2.00000e+00, 1.00000e+00, 1.50000e+01,
        1.05000e+03, 1.42900e+01],
       [5.00000e+00, 1.70000e+01, 2.00000e+00, 1.00000e+00, 2.70000e+01,
        2.09100e+03, 1.29100e+01],
       [5.00000e+00, 2.20000e+01, 2.00000e+00, 1.00000e+00, 1.30000e+01,
        9.16000e+02, 1.41900e+01],
       [5.00000e+00, 2.20000e+01, 2.00000e+00, 1.00000e+00, 1.70000e+01,
        1.23800e+03, 1.37300e+01],
       [5.00000e+00, 2.20000e+01, 3.00000e+00, 1.00000e+00, 2.00000e+01,
        2.00000e+01, 1.00000e+03],
       [5.00000e+00, 2.20000e+01, 3.00000e+00, 1.00000e+00, 3.50000e+01,
        2.45800e+03, 1.42400e+01],
       [5.00000e+00, 2.20000e+01, 3.00000e+00, 2.00000e+00, 1.60000e+01,
        8.75000e+02, 1.82900e+01],
       [5.00000e+00, 2.20000e+01, 3.00000e+00, 2.00000e+00, 5.10000e+01,
        3.33500e+03, 1.52900e+01],
       [5.00000e+00, 2.20000

In [76]:
#X=ML.iloc[:,[1,4,5,6]].values
#y=ML.iloc[:,[0,2,3]].values
#y

In [77]:
#Normalizing the data
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)


In [78]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
y = ohe.fit_transform(y).toarray()
y

C:\Users\kahme\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


array([[0., 1., 1., ..., 1., 1., 0.],
       [0., 1., 1., ..., 1., 1., 0.],
       [0., 1., 1., ..., 1., 1., 0.],
       ...,
       [0., 1., 1., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 1., 1., 0.]])

In [79]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.1)

In [80]:
from sklearn.preprocessing import StandardScaler

# Create a StandardScater model and fit it to the training data
X_scaler = StandardScaler().fit(X_train)

In [81]:
# Transform the training and testing data using the X_scaler

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [82]:
from tensorflow.keras.utils import to_categorical

In [83]:
# One-hot encoding
y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)

In [88]:

# Neural network

model = Sequential([
    Dense(16, input_dim=7),
    Activation('relu'),
   
    Dense(4),
    Activation('softmax'),
])

In [89]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 16)                128       
_________________________________________________________________
activation_9 (Activation)    (None, 16)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 4)                 68        
_________________________________________________________________
activation_10 (Activation)   (None, 4)                 0         
Total params: 196
Trainable params: 196
Non-trainable params: 0
_________________________________________________________________


In [90]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [91]:
history = model.fit(X_train, y_train, epochs=2
                   )

ValueError: Error when checking input: expected dense_9_input to have shape (7,) but got array with shape (4,)